socket编程

In [ ]:
import socket

# tcp
sk_tcp = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
#upd
sk_udp = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)


sk_tcp.close()

sk_udp.close()

关于线程的使用

In [7]:
import threading


def say_sth(sth,num):
    print(sth)
    print(num)
    
    
#创建线程并启动
t = threading.Thread(target=say_sth,args=([1,2,3],10))
t.start()

# #获取当前(进程)所有线程
# print(threading.enumerate())



class YourClass(threading.Thread):
    #继承Thread类必须实现run方法，才能通过start方法来启动
    def run(self):
        print("do your job")


        
# t = YourClass()
# t.start()


[1, 2, 3]
10


使用锁解决资源竞争问题

In [19]:
import threading
g_number = 0

def method1(num):
    global g_number
    mutex.acquire()
    for i in range(num):
        g_number += 1   
    print("g_number in method1 is ",g_number)
    mutex.release()
    
def method2(num):
    global g_number
    mutex.acquire()
    for i in range(num):
        g_number += 1
    print("g_number in method2 is ",g_number)
    mutex.release()
    
mutex = threading.Lock()
t1 = threading.Thread(target=method1,args=(1000000,))
t2 = threading.Thread(target=method2,args=(1000000,))
t1.start()
t2.start()

t1.join()
t2.join()
print("g_number in main thread is ",g_number)

g_number in method1 is  1000000
g_number in method2 is  2000000
g_number in main thread is  2000000


上锁的代码要尽可能少

In [21]:
import threading
g_number = 0

def method1(num):
    global g_number
    for i in range(num):
        # 上锁的代码要尽可能少
        mutex.acquire()
        g_number += 1   
        mutex.release()
    print("g_number in method1 is ",g_number)
    
    
def method2(num):
    global g_number
    
    for i in range(num):
        mutex.acquire()
        g_number += 1
        mutex.release()
    print("g_number in method2 is ",g_number)
    
    
mutex = threading.Lock()
t1 = threading.Thread(target=method1,args=(1000000,))
t2 = threading.Thread(target=method2,args=(1000000,))
t1.start()
t2.start()

t1.join()
t2.join()
print("g_number in main thread is ",g_number)

g_number in method1 is  1951472
g_number in method2 is  2000000
g_number in main thread is  2000000


队列的使用----多进程共享数据

In [2]:
import multiprocessing
q = multiprocessing.Queue()  # 设定队列大小，一般不指定
q.put(2) # 如果满了会阻塞
q.put_nowait(3) # 不阻塞，满了会报异常
q.get() # 如果空会阻塞
q.get_nowait()  # 空了不会阻塞，但会报错
q.empty() # 空判断
q.full()  # 满判断



In [4]:
import multiprocessing
def down_load_data(queue):
    # 模拟从网络下载数据
    data = [1,2,3,4,5]
    for _ in data:
        queue.put(_)
    print("data is ok")
    

def analysis_data(queue):
    # 模拟分析数据
    data = list()
    while not queue.empty():
        data.append(queue.get())
    print("data analysis ok")
    print(data)

def main():
    queue = multiprocessing.Queue()
    p1 =  multiprocessing.Process(target=down_load_data,args=(queue,))
    p2 =  multiprocessing.Process(target=analysis_data,args=(queue,))
    

    p1.start()
    p2.start()
    
if __name__ == "__main__":
    # jupyter 的原故，这里起动进程没有什么显示，打代码放到pychar就有结果了
    main()


进程池

In [1]:
import time
import os
import multiprocessing

def work(index):
    print("proccess：{}, index：{} is finished".format(os.getpid(), index))
    time.sleep(3)


if __name__ == "__main__":
    """
    在windows中, 创建进程由于没有folk创建进程, 所以是通过重载自身模块来创建的, 
    如果Pool放在if __name__ == "__main__"外部, 就会进行无限递归
    
    代码在jupyter中没有效果
    """
    pool = multiprocessing.Pool(3)  # 最多开辟3个进程
    for i in range(10):
        pool.apply_async(work, (i,))  # 向池子添加任务
#     pool.close()  # 关闭进程池，不再接受新的请求
#     pool.join()  # 等待所有子进程运行结束,必须放在close之后
    print("------end------")
    

------end------


多进程的queue、锁

import queque 普通的队列，在多进程中没用
multiprocessing.Queue()  可在多进程中使用，但不可以在进程池中使用
multiprocessing.Manager().Queue()  可以在进程池中使用

多进程的锁也是一样
multiprocessing.Manager().Lock()  可以在进程池中使用

迭代器

In [2]:
# 一个对象是否可迭代，主要看是否派生自Iterable
from collections import Iterable
print(isinstance("abcd",Iterable))

# 要实现一个可迭代的对象，必须实现__iter__方法，__iter__方法的返回值就是一个迭代器，要实现一个迭代器，又必须同时实现__iter__和__next__方
# 法

class Workmate(object):
    
    def __init__(self):
        self.names = list()
        self.index = 0
    def add(self,name):
        self.names.append(name)
        
    def __iter__(self): # 会被 iter()方法自动调用，遍历的时候也会被自动调用，返回一个可迭代的对像，Workmate自身就是可迭代的
        return self
    
    def __next__(self):   # 会被 next()方法自动调用，注意这里会存在下标越界问题，要判断是否小于len
        if self.index < len(self.names):
            ret = self.names[self.index]  
            self.index += 1
            return ret
        else: # 在循环中，__next__会一直被调用，就算下标越界了，它也不知道什么时候停止，所以一定要抛出StopIteration异常，for循环才会停
            raise StopIteration



workmate = Workmate()
workmate.add("张三")
workmate.add("李四")
workmate.add("王五")
    
for temp in workmate:
    print(temp)

True
张三
李四
王五


生成器，是一种特殊的迭代器
方式1 把列表推导式的中括号换成小括号，就得到了一个生成器   

In [9]:
ls = [x for x in range(10)]  # 这种形式很占空间
temp =(x for x in range(10)) # 这种形式不占空间
for num in temp:
    print(num)

0
1
2
3
4
5
6
7
8
9


方式2 只要方法里面有yield，就是一个生成器

如果这种方式生成的生成器里面有return一个结果，通常这个结果是拿不到的，通过next()一直迭代下去，到最后会报异常，try之后拿到异常对像ex
通过ex.value可以拿到return的结果

In [2]:
def fibonacii(number):
    """用生成器来生成斐波那契数列"""
    a, b = 0, 1
    counter = 0
    while counter < number:
        yield a
        a, b = b, a + b
        counter += 1
        
temp = fibonnacii(10)  # 这里就变成了创建生成器，而不是调用普通函数
for _ in temp:
    #print(_)
    pass

除了next()方法可以调用用yield生成的生成器，send()也可以调用生成器，并且可以向生成器传入参数。但，send()方法首次调用生成器传参只能传None，否则会报错。

In [8]:
def fibonacii(number):
    """用生成器来生成斐波那契数列"""
    a, b = 0, 1
    counter = 0
    while counter < number:
        ret  = yield a
        print("ret = ", ret)
        a, b = b, a + b
        counter += 1
        
        
obj = fibonacii(10)
# print(obj.send("your args"))  会报错，因为没有任何变量去接send()传入的参数
print(next(obj)) 
result = obj.send("hello world") 
# 在第一次调next()之后，程序在yield a处将值传出并暂停，send将参数传入给里面的ret，然后程序继续往前走，
# (你可以用拿到的参数修改原业务，不修改也可以)。在下一次yield a的时候，参值返回给外面，同时将a的值赋给ret，所以send的返回值是1
print(result)


0
ret =  hello world
1


In [ ]:
用yield实现多任务(假的多线程)

In [ ]:
import time
def task_1():
    while True:
        print("--------task_1-----------")
        time.sleep(1)
        yield # 重点是这里
        
def task_2():
    while True:
        print("--------task_2-----------")
        time.sleep(1)
        yield
        
def main():
    t1 = task_1()
    t2 = task_2()
    while True:
        next(t1)
        next(t2)
    

greenlet实现多任务--知道可以这么用而矣

In [3]:
from greenlet import greenlet
import time

def task1():
    while True:
        print("task1")
        g2.switch()
        time.sleep(1)
        
def task2():
    while True:
        print("task2")
        g1.switch()
        time.sleep(1)
        
g1 = greenlet(task1)
g2 = greenlet(task2)
# 切换到g1中运行
g1.switch()


gevent实现多任务（重点）,gevent是对greenlet的进一步封装！  
gevent一遇到耗时操作(gevent.sleep(1))，就会马上切换到下一个任务。  
如果不想用gevent.sleep()，要给代码先打包。

In [2]:
import gevent
import time
from gevent import monkey
import random

# 打包代码，将程序中所有耗时操作替换为gevent自己的实现，如time.sleep(1)会变成 gevent.sleep(1)
monkey.patch_all()

def task(n):
    for i in range(5):
        print(n, i)
        time.sleep(0.5)

# joinall 启动任务并等等所有任务运行完 如C#中的Task,WaitAll是一个意思
gevent.joinall([ 
    gevent.spawn(task,"task1"),# 新建任务并启动
    gevent.spawn(task,"task2"),
    gevent.spawn(task,"task3")
])


python中的属性

In [ ]:
class Person(object):
    def __init__(self, age):
            self.__age = age
    
    @property
    def age(self):
        return self.__age
    
    @age.setter
    def age(self,value):
        self.__age = value
        
    @age.deleter   # 删除属性，一般不写，除非有特征需求
    def age(self):
        del self.__age
    
    
p = Person(20)
p.age
p.age = 30
p.age
# del p.age
# p.age

In [12]:
# 这种实现，get_age、set_age 方法是可以通过实例直接调用
class Person(object):
    def __init__(self, age):
            self.__age = age
    
    def get_age(self):
        return self.__age
    
    
    def set_age(self,value):
        self.__age = value
    
    age = property(get_age, set_age, None, "doc_string of age property") # 第三个参数为删除属性的方法
    
p = Person(20)
p.age
p.age = 30
# p.set_age(50)
p.age

with---上下文管理器：只要实现了 __enter__()、__exit__()方法的类型，就是上下文管理器

In [24]:
class MyWith(object):
    """自定义上下文管理器"""
    def __init__(self, file_name, mode):
        self.file_name = file_name
        self.mode = mode
        
    def __enter__(self):
        self.f = open(self.file_name, self.mode)
        return self.f
    
    def __exit__(self, type_, value, stack_trace): # 后面3个参数与处理异常有关，暂时不展开
        print("退出方法被调用")
        self.f.close()
        
with MyWith("Interview.txt", "r") as f:
    print(f.readline())
    

退出方法被调用


In [23]:
# 更简捷的实现方式
from contextlib import contextmanager
@contextmanager
def my_with(path, mode):
    f = open(path, mode)
    yield f
    f.close()
    
with my_with("Interview.txt", "r") as f:
    print(f.readline())

In [ ]:
"""
通过字符串变量导入模块
temp ="your_module"
通过一个字符串变量名来表示导入的模块名，用import来导入，肯定会报错
import temp  # 会变成程序以为要导入 temp模块
这种情况要通过  __import__(temp) 来导入
module =  __import__(temp) # module  就是temp所要的模块
通过 getattr()可以获取模块中的成员
func = getattr(module, "function_name")
func()  # 可以调用
"""

In [ ]:
# 闭包---直接方程 y = kx + b
# 闭包通常是一个函数里面套一个(或多个)函数，返回值是里面的函数，而里面的函数通常会使用到外面函数的参数
# 里面函数的整个空间，就形成一个闭包，闭包会将外面函数的形参给固定下来

# 函数与闭包的区别：函数只有对数据处理的逻辑，没有数据，而闭包包含了对数据处理的逻辑以及数据

def line(k, b):
    def y(x):
        print(k * x +b)
    return y

l = line(2, 3)  #传入的 2,3会被固定在内容函数y里面
l(3)


# 想要修改闭包外面的函数的局部变量，要用nonlocal

def test1():
    x = 200
    def test2():
        nonlocal x  #  python2 没有nonlocal
        print("-------1-------x=%d" % x)
        x = 100 # 修改
        print("-------1-------x=%d" % x)
    return test2

t1 = test1()
t1()


装饰器的演变过程

In [30]:
#装饰器演变过程1
#  4(即x参数)已经被固定在闭包中了,并且new_add的返回值是一个函数
# def add(a,b):
#     return a+b

# def new_add(x):
#     def inner(y):
#         return x + y
#     return inner


# foo = new_add(4)
# ret = foo(5)
# print(ret)

        
#装饰器演变过程2
#把add函数作为参数，传给了logger函数，logger函数用wrapper对add进行了加工(没有改变add的原有功能)，且logger的返回
#值是一个函数wrapper,而wrapper内部是肯定会执行add函数的

# def looger(fn):
#     def wrapper(*args, **kwargs):
#         print("mark the log")
#         return fn(*args, **kwargs)
#         #print("or do sth")
#     return wrapper
    
# def add(a,b):
#     return a+b

# foo = looger(add)
# print(foo(10,10))


#装饰器演变过程3
#将add 传给logger的时候，add已经被固定在logger的wrapper中了，然后logger返回的是一个函数，也就是左边的add已经是一个被增强过的add了

# add = looger(add)  # 但是这样写很low
# add(10,20)
            
    
#装饰器演变过程4
def logger(fn):
    def wrapper(*args, **kwargs): # 这里是定义有可变参数，写可变参数只是为了方便点，也可以随机定义形参与add参数对应就可以了
        print("mark the log")
        return fn(*args, **kwargs) # 这里是把可变参数解出来，传给fn，即add(a, b)
        #print("or do sth")
    return wrapper

@logger  # 等价于： add = logger(add)
def add(a,b):
    return a + b    


print(add(10,20))

mark the log
30


In [32]:
# 用装饰器去装饰类，可实现修改类的功能
def set_property():
    def wrapper(cls):
        cls.name = None  # i添加了一个类变量 name
        return cls
    return wrapper
    
    
@set_property()
class Person:
    def __init__(self):
        pass
        

In [49]:
# 用类作为装饰器，可以利用__call__ 方法，对数据进行加工，使用场景少，了解即可。

class Person():
    def __init__(self, fn):
        self.fn = fn
        
    def __call__(self, *args, **kwargs):
        # 本列中没有加工，可以添加加工的逻辑
        return self.fn(*args, **kwargs)

@Person
def add(a, b):
    return a + b

ret = add(10, 20) # 相当于调用  __call__()
print(ret)

30


In [8]:
# 带参数的装饰器
def logger(level):
    def log(fn):
        def wrapper(*args, **kwargs): 
            if level == 1:
                print("mark the log")
            elif level == 2 :
                print("check authoright")
            return fn(*args, **kwargs)
        return wrapper
    return log  # logger的返回值

@logger(2)  # 这里就变成了两步：1、调logger并传递实参；2、拿到logger的返回值(即log)对add函数进行装饰
def add(a,b):
    return a + b   

print(add(10,20))

check authoright
30


In [14]:
# 解耦因参数不一样，调用的方法不一样

def test01():
    pass

def test02():
    pass

def test03():
    pass


def main():
    
    func_name ="test01" # input("请输入")
    if func_name == "test01":
        test01()
    elif func_name == "test01":
        test02()    
    elif func_name == "test01":
        test03()
    else:
        pass
    
    # 这样写在实现上没有问题，但如果有100个函数，就有100个if
    
    

    
def test04():
    pass

def test05():
    pass

def test06():
    pass    
 
    
    
FUNCTIONS = {
    'test04':test04,
    'test05':test05,
    'test06':test06
}

def main():
    #func_name ="test04" # input("请输入")
    #FUNCTIONS[func_name]() #用字典存起函数引用，将来索引调用；缺点：当key没有的时候，肯定崩！而且每加一个函数，要手动悠入字典
    pass
    
    
    
# 用带参数的装饰器来实现函数解耦
FUNCTIONS = dict()
def route(name):
    def wrapper(fn):
        # 在这里已经可以拿到被装饰的那个函数的引用(即fn)
        FUNCTIONS[name] = fn
        def inner(*args, **kwargs):
            return fn(*args **kwargs)
        return inner
    return wrapper

@route("a")
def test07():
    print("test07")

@route("b")
def test08():
    print("test08")

@route("c")
def test09():
    print("test09")

def main():
    func = "b" # input("请输入")
    # 通过装饰器，自动的把函数引用放到字典中，而且key不必与函数名一样。如果没有对应的key，还是会崩，这时用if 可 try都可以解决
    # 推荐使用异常，因为，异常可以传递，try不单单可以处理key不存在的问题，如果对应的函数里有异常，也可以try住
    #if func in FUNCTIONS:
        #FUNCTIONS[func]()
    
    try :
        FUNCTIONS[func]()  
    
    except Exception as ex:
        print("有异常")
    
    
    

main()

test08


In [7]:
# log日志使用
import logging

logging.basicConfig(level=logging.INFO)

# logging.debug("this is debug msg")
# logging.info("this is info msg")
# logging.warning("this is warning msg")
# logging.error("this is error msg")
# logging.critical("this is critical msg")

# 像上面那样写，基本没有什么用，要加上格式
format_str = '%(asctime)s - %(filename)s[line:%(lineno)d] - %(levelname)s: %(message)s'
logging.basicConfig(level=logging.INFO,
                    format=format_str
                    )

# logging.debug("this is debug msg")
# logging.info("this is info msg")
# logging.warning("this is warning msg")
# logging.error("this is error msg")
# logging.critical("this is critical msg")

# 将日志写入到文件
logging.basicConfig(level=logging.INFO,
                    format=format_str,
                    filename='./log.txt',
                    filemode='w'
                    )

# 既显示到终端，也写入到日志文件
# 1、创建一个logger
logger = logging.getLogger()
logger.setLevel(logging.INFO) # 设置记录级别

# 2、 创建一个handler，用于写入日志文件
logfile = '.log.txt'
fh = logging.FileHandler(logfile,mode="a") # 以追加形式添加
fh.setLevel(logging.DEBUG) # 输出到file的日志等级

# 3、再创建一个handler用于输出到控制台
ch = logging.StreamHandler()
ch.setLevel(logging.WARNING) # 设置输出到控制台的日志的级别

# 4、定义handler的格式
formatter = logging.Formatter(format_str)
fh.setFormatter(formatter)
ch.setFormatter(formatter)

# 5、将handler添加到logger里面
logger.addHandler(fh)
logger.addHandler(ch)

# 记录日志
logger.debug("this is debug msg")
logger.info("this is info msg")
logger.warning("this is warning msg")
logger.error("this is error msg")
logger.critical("this is critical msg")



In [ ]:
# 元类
# 用元素创建一个类，有几个类变量
Person = type("Person",(),{"id":1,"age":10,"name":"tom"})  # 中间的元组是继随，没有就写空元组


In [34]:
# ORM实现核心

class BaseModel(type):
    def __new__(cls, name, bases, attrs):
        mappings = dict()
        for k, v in attrs.items():
            if isinstance(v, tuple):
                mappings[k] = v  # 保存实体类中的属性映射关系 如 uid = ('uid', "int unsigned")
    
        # 删除实体类中定义的属性，到时再用setattr将属性挂在实例上，就可以做到实体类的属性，与表字段名不必一样
        for k in mappings.keys():
            attrs.pop(k)

        attrs["__mappings__"] = mappings
        attrs["__table__"] = name
        return type.__new__(cls, name, bases, attrs)
    

class Model(object, metaclass=BaseModel):
    """实体类的基类，将增删改查都抽象到这里"""
    def __init__(self, **kwargs):
        # 将属性挂在实体上
        for name, value in kwargs.items():
            setattr(self, name, value)
            
    def save(self):
        fields = []
        args = []
        for k, v in self.__mappings__.items():
            fields.append(v[0]) # 取的时候从映射关系中取，取字段元素的第一个元素，即字段名
            args.append(getattr(self, k, None)) # 字段值
        
        temp_args = list() # 处理数据不加引号，字符串加引号的问题
        for item in args:
            if isinstance(item, int):
                temp_args.append(str(item))
            elif isinstance(item, str):
                temp_args.append("'%s'" % item)
        sql = "insert into %s(%s) values(%s)" % (self.__table__, ','.join(fields), ','.join([str(i) for i in temp_args]))
        #print("sql：", sql)    
    
    def update(self):
        pass
    
    def delete(self):
        pass

class User(Model):
    # 定义属性及映射关系，最终会去到 __mappings__字典中
    uid  = ('uid', 'int unsigned')
    name = ("username", "varchar(30)")  # 属性名可以与字段名不一致
    email = ("email", "varchar(30)")
    password = ("password", "varchar(30)")
    

       
u = User(uid=123, name="tom", email="tom@tom.com",password="123456")  


In [ ]:
"""
django的使用

创建一个django项目:
django-admin startproject prject_name

目录文件说明：
__init__.py  说明 prject_name 是一个 python 模块
settings.py  项目的配置文件
urls.py  进行url路由的配置
wsgi.py  web服务器和django交互的入口
manage.py 项目的管理文件



创建应用：需要进入到项目目录
python manage.py startapp app_name

应用的目录文件说明：
__init__.py  说明 app_name 是一个 python 模块
models.py 写和数据库项目的内容
views.py 接收请求，进行处理，与 M 和 T进行交互，返回应答
test.py  写测试代码的文件
admin.py  网站后台管理相关的文件


建立应用和项目之间的联系

修改settings.py 中的 INSTALLED_APPS配置项，要配置项的最后，写上你的app_name就可以了
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'booktest' # 你的app name
]



运行django的开发服务器
python manage.py runserver

生成迁移文件（要先定义好实体类）
python manage.py makemigrations [app_name] # app_name 可以不写

注意：django 2.x后，如是实体类中有外键属性，要写上on_delete=models.CASCADE表示级联删除
如： hbook = models.ForeignKey("BookInfo", on_delete=models.CASCADE)
否则迁移的时候会报  __init__() missing 1 required positional argument: 'on_delete' 错误


执行迁移生成数据库表
python manage.py migrate


通过实体类操作数据表：
进入项目 shell的命令
python manage.py shell

首先导入模型
from booktest.models import BookInfo

创建实体类对象
class BookInfo(models.Model):
    btitle = models.CharField(default=20)
    bpub_date = models.DateField()
    bread = models.IntegerField(default=0)
    bcomment = models.IntegerField(default=0)
    isDelete = models.BooleanField(default=False)
    
class HeroInfo(models.Model):
    hname = models.CharField(default=20)
    hgender = models.BooleanField(default=False)
    hcomment = models.CharField(default=200)
    hbook = models.ForeignKey("BookInfo", on_delete=models.CASCADE)
    isDelete = models.BooleanField(default=False)
    

b = BookInfo()
b.btitle="天龙八部"
b.bpub_date = date(1990, 10, 10) 需要从datetime模块导入 date
b.save() 写到数据库




由一查多  “多”的那个类名小写
b.heroinfo_set.all()

多查一
h = HeroInfo()
h.hbook  # 外键属性




【后台管理】
1、本地化
语言和时区本地化，在settings.py文件中改
LAMGUAGE_CODE = 'zh-hans'
TIME_ZONE = "Asia/Shanghai"

2、创建管理员
命令：python mamage.py createsuperuser
按提示填写账号、邮箱(测试的不一定要真实)、密码
要登录的话，就可以启动测试服务器
python manage.py runserver # 后面可以跟端口来指定端口
地址：http://127.0.0.1:8000/admin


3、注册模型类
在应用下的 admin.py 中注册模型类
告诉 django 框架根据注册的模型类来生成对应表管理页面
导入模型类
from bootest.models import BookInfo
admin.site.register(BookInfo)  # 注册要管理的模型

为了在表管理页面中显示好看，需要在模型类中重写 __str__(self)方法,否则会显示XXXobject
如
def __str__(self):
    return self.btitle

4、自定义管理页面，要显示的内容(在admin.py中定义模型管理类，要注册之前)
class BookInfoAdmin(admin.ModelAdmin): # 一般管理类与模型类起名要对应
    list_display = ['id','btitle','bpub_date']


注册时写多一个参数
admin.site.register(BookInfo, BookInfoAdmin)






【视图的使用】
1、定义视图函数 在 views.py中

如：
from django.http import HttpResponse

# 请求的时候 http://127.0.0.1:8000/index 会进入这个方法
def index(request):
    # 业务代码
    return HttpResponse("hello python")


2、进行url配置，
2.1、在 应用(booktest) 的目录下建 urls.py文件

2.2 导入视图文件、导入相关方法
from booktest import views
from django.conf.urls import url   #django 1.x 
from django.urls import path  # django 2.x

2.3 定义一个url路由配置列表
1.x 写法
urlpatterns = [
    url(r"^index", views.index), # 建立/index和视图index之间的关系
]

2.x写法
urlpatterns = [
    path("index", views.index), # 建立/index和视图index之间的关系
]


2.4 再到项目的urls.py中包含bootest应用中的urls文件

1.x 写法
from django.conf.urls import url, include
urlpatterns = [
 url(r"^admin/", include(admin.site.urls)),
 url(r"^", include("booktest.urls"))
]

2.x 写法
from django.urls import path, include
from booktest import views
urlpatterns = [
  path('admin/', admin.site.urls),
  path('index', include("booktest.urls"))
  
]

注意：如果视图文件中有两个名字相近的视图，url规则要注意匹配方式

如：
def index2(request):
    return return HttpResponse("hello world")


应用的 urls.py就要写清楚匹配规则
urlpatterns = [
    url(r"^index$", views.index), # 建立/index和视图index之间的关系
    url(r"^index2$", views.index2)
]




【模板的使用】
1、在项目目录下创建一个 templates 目录
2、支项目的setting.py文件中配置 TEMPLATES 下的 DIRS 注意：不要直接写定物品路径，配置文件中有个变量 BASE_DIR
   获取了项目的路径，"DIRS":[os.path.join(BASE_DIR, 'templates')]

3、一般一个应用的模板文件放到一起方便管理，所以在template目录下，再将一个与应用同名的目录(booktest)，
   表示这些都是某个应用的模板

4、在模板目录booktest下，建index.html模板文件
  4.1 加载模板文件（回到视图文件中） 去模板目录下面获取html文件的内容，得到一个模板对象
      from django.template import loader, RequestContext
      temp = loader.get_template("booktest/index.html")
      
  4.2 定义模板上下文，向模板文件传递数据
      #context = RequestContext(request, {}) #以字典形式将数据传到模板，不传写空字典 经测试无效，注释
      
  
  4.3 模板渲染  得到一个标准的html内容
      # res_html = temp.render(context) 改为
      res_html = temp.render(context=locals(), request)
      
  4.4 返回内容给浏览器
      return HttpResponse(res_html)


但是，以上几个使用模板的步骤，紧知道怎么回事即了，django已经封装好了方，直接调用即可
    return render(request, "booktest/index2.html")
    

模板文件中使用变量的格式：  {{变量名}}
传值：return render(request, "booktest/index2.html", {"变量名":变量值,"list":list(range(10)))

模板文件中也可以写  类似  后端的代码，
<ul>
    {% for i in list %}
    <li>{{i}}</li>
    {% endfor %}
</ul>


<ul>
    {% for hero in heros %}
    <li>{{hero.hname}}---{{hero.hcomment}}</li>
    {{% empty %}} # 如果上面为空，则显示下面
        <li>没有英雄信息</li>
    {% endfor %}
</ul>


如果视图方法中有参数，那么在应用的urls.py文件中的url规则中，要有体现
def detail(request, bid):
    pass
    
urlpatterns = [
    ...
    url(r"^books/(\+d)$", views.detail)  # 正则分组的数据会传到detail方法的bid中
]




【django项目启用mysql】

去项目的setting.py里面配置 DATABASES

DATABASES = {
    'default': {
        # 'ENGINE': 'django.db.backends.sqlite3',
        # 'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
        'ENGINE': 'django.db.backends.mysql',
        'NAME': "pythontest",  # 数据库名字，库要手动先建好
        "USER": "root",  # 用户名
        "PASSWORD": "root",  # 密码
        "HOST": "192.168.43.100",  # 服务器IP
        "PORT": 3306  # 端口
    }

启动django项目时，如果出现 Error loading MySQLdb module:No module named "MySQLdb"，说明没装pymysql
安装之后，再到项目的 __init__.py文件中加入下面两句代码
import pymysql
pymysql.install_as_MySQLdb()



【在视图中重定向】
from django.http import HttpResponseRedirect
在视图中
return HttpResponseRedirect("/index/")  # 参数为你想要重定向的路径

如果想要简写：
from django.shortcuts import render, redirect
return redirect("/index/")




【模型字段属性、选项】
1、想要某个属性在数据表里面是唯一的，加上 unique = True
2、想要为某个属性建索引，加上 db_index = True
3、想为属性名指定字段名，加上 db_column = "field name"
4、是否允许为空 null = True # 对于数据库表来产的
5、某个属性在表单上，是否允许为空(字符串) blank = True


【查询的相关函数】
get：返回一条且只能有g条数据，返回值是一个对象，参数可以写查谒条件。
all：返回模型类对应表的所有数据，返回值是QuerySet。
filter：返回满足条件的数所d，返回值是QuerySet，参数可以写查询条件。
exclude：返回不满足条件的数据。
order_by：对查询结果进行排序。

F对象：用于类属性之间的比较
Q对象：有t于条件之间的逻辑关系。

aggregate：进行聚合操作，返回一个字典
count()：返回结果集中数据的数目
exists():判断查询集是否有数据

注：
通过模型为d实现关联查询时，要查哪个表中的数据，就需要通过哪个类来查。
写关联查询条件是f，如果类中没有关系属性，条件需要写对应的类的类名，如果类中有关系属性，直接写关系属性。

【条件查询】
filter() 里面写多个条件，默认是 【与】的关系
BookInfo.objects.filter(id__gt=3, bread_gt=30) # id>3 and bread >30

想使用【或】查询，要使用Q对象
from django.db.models import Q
Q对象使用 & | ~  表示与或非


BookInfo.objects.filter(Q(id__gt=3)|Q(bread_gt=30)) # id>3 or bread >30
BookInfo.objects.filter(Q(id__gt=3)&Q(bread_gt=30))  #也可以且

BookInfo.objects.filter(~Q(id__gt=3) # ~ 取反，就不大于3


如果查询条件是属性值与属性值之间的比较，需要使用 F对象
from django.db.models import F

BookInfo.objects.filter(bread__gt=F("bcommnet")) # 阅读量大于评论量 
BookInfo.objects.filter(bread__gt=F("bcommnet") * 2 ) # 阅读量大于2倍评论量 



聚合函数：
使用前需要先导入聚合类
from django.db.models import Sum,Count,Max,Min,Avg

BookInfo.objects.all().aggregate(Count("id")) # 统计，Count里面不能写*，要写明哪个字段，all可以省略

统计还是可使用 count函数(大写的Count 是类型)统计数据
BookInfo.objects.count()
BookInfo.objects.filter(id__gt=3).count()  # 大于3的条数


"""


In [ ]:
"""
【通过模型类实现关联查询】

查询图书信息，要求图书关联的英雄的描述包含 ‘八’。
BookInfo.objects.filter(heroinfo__hcomment__contains='八')

查询图书信息，要求图书中的英雄的id大于3
BookInfo.objects.filter(heroinfo__id__gt=3) # BookInfo中没有关系属性

查询书名为‘天龙八部’的所有英雄
HeroInfo.objects.filter(hbook_btitle='天龙八部') # HeroInfo中有关系属性 hbook



自关联模型类
class AreaInfo(models.Model):
    atitle = models.CharField(max_length=20)
    # 代表当前地区的父级地区
    aParent = models.ForeignKey('self', null=True, blank=True)





【管理器】objects 是models.Manager类的一个实体，自定义管理器后，django就不再生成默认的objects管理器。
自定义管理器一般用于：1、改变查询结果集；2、添加额外的方法；


如：在BookInfo类中定义一个属性  book = models.Manager()，objects就会消失，这样写是没有什么 意义的，
一般是重新定义一个类，继承mdels.Manager类，然后重写里面的方法，实现自己想要的功能(如all()方法返回的是isDelete=0的数据)。

class BookInfoManager(models.Manager):
    '''图书模型管理器类'''
    
    def all(self): # 改变查询结果集
        books = super().all().filter(isDelete=False)
        return books
       
    # 添加方法，操作模型类对应的数据表进行增删改查，这样的方法django已经写好objects.create()
    def create_book(self, btitle,bpub_date): 
        # book = BookInfo()  # 这样写，如果模型类改名，是要一起改的，麻烦。django已经封装好属性解决这个问题
        model_class = self.model # 就是对应的模型类
        book = model_class()
        
        
        book.btitle = btitle
        book.bpub_date = bpub_date
        book.save()
        return book
        

class BookInfo(models.Model):
    ...
    objects = BookInfoManager() # objects 已经被替换成自定义的实现
    
    # 【元选项】：用于指定模型类生成的表名，这样生成的表名，就不再是 appname_modelname 这种形式
    class Meta:
        db_table = "your table name"



【关于调试模式】
发布前把调试模式设置为False，设置之后，ALLOWED_HOST = ["*"]

【关于404】
在关闭调试模式之后，请求不存在的页面时，会返回一个默认的404页面，当你想用自定义的404页面时，在应用的模板目录下
建一个404.html即可，在404页面中，django会传过一个叫 request_path 的变量，代表用户请求的页面
同理，500错误也可以建一个500.html页面

【视图的功能】
1、返回HttpResponse
2、返回redirect
3、返回JsonResponse   #需要导入，在HttpResponse相同的包中 


【捕获url参数】
url(r'delete(\d+)', view.delete) # 捕获url参数：位置参数， 在视图中随机定义一个形参接收都可以
url(r'delete(?P<num>\d+)', views.delete) # 捕获url参数：关键字参数，视图中的形参必须是与分组名(?P是给分组命名)相同


【request对象】
request.Post 保存的是post提交的参数，是一个QueryDict类型
request.Get  保存的是get提交的参数

QueryDict使用
q = QueryDict("a=1&b=2&c=3") # 创建了3个键值对
q['a'] # 1  # 键不存在会报错
q.get('b') # 2 键不存在，不会报错
q.get('b', '指定默认值')

q = QueryDict("a=1&a=2&a=3") # 写字典有点不一样，这个可以键名相同
q.get('a') # 3  总是取最后一个
q.getlist('a') 返回一个列表 [1,2,3] 有点像url传同名键会变成数组的意思



如果Post提交的时候，出现403错误，去项目的配置文件，把47行 django.middlewar.csrf.CsrfViewMiddleware 注释掉
form 表单
<form method="post" action="/login_check/">
    用户名：<input type="text" name="username"/><br/>
    密码：<input type="password" name="password"/><br/>
    <input type="submit" value="登录">
</form>

视图中取参数
username = request.POST.get('username')
password = request.POST.get('password')



HttpReqeust对象
•服务器接收到http协议的请求后，会根据报文创建HttpRequest对象
•视图函数的第一个参数是HttpRequest对象
•在django.http模块中定义了HttpRequest对象的API

属性
•下面除非特别说明，属性都是只读的
•path：一个字符串，表示请求的页面的完整路径，不包含域名
•method：一个字符串，表示请求使用的HTTP方法，常用值包括：'GET'、'POST'
•encoding：一个字符串，表示提交的数据的编码方式◦如果为None则表示使用浏览器的默认设置，一般为utf-8
◦这个属性是可写的，可以通过修改它来修改访问表单数据使用的编码，接下来对属性的任何访问将使用新的encoding值

•GET：一个类似于字典的对象，包含get请求方式的所有参数
•POST：一个类似于字典的对象，包含post请求方式的所有参数
•FILES：一个类似于字典的对象，包含所有的上传文件
•COOKIES：一个标准的Python字典，包含所有的cookie，键和值都为字符串
•session：一个既可读又可写的类似于字典的对象，表示当前的会话，只有当Django 启用会话的支持时才可用，详细内容见“状态保持”

方法
•is_ajax()：如果请求是通过XMLHttpRequest发起的，则返回True


"""

In [ ]:
"""
【使用静态文件】
js css images

1、在项目的目录下建一个叫 static 目录，再在里面建3个目录  js css images
2、去项目的settings.py文件中加一个 SATICFILES_DIRS = [os.path.join(BASE_DIR, "static")]

把js文件放到对应的目录，在html文件中通过 script 标签引入，就可以使用js文件了


【在模板中引用静态文件--官方文档介绍的写法】

{% load staticfiles %}
<script src="{% static 'js/jquery-1.12.4.min.js' %}"></script>
<script>
    $(function () {
        $("#btnLogin").click(() => {
            let username = $("#username").val();
            let password = $("#password").val();
            alert(username);
        });
    })
</script>

如果不想麻烦，每次都写下面两句，可以 SATICFILES_DIRS = [os.path.join(BASE_DIR, "static"),'/static/'] 加多一个路径'/static/'
{% load staticfiles %}
<script src="{% static 'js/jquery-1.12.4.min.js' %}"></script>

就可以写 <script src="/static/js/jquery-1.12.4.min.js"></script>，不确定发布会不会出问题，反正调试OK


STATIC_URL = '/static/'  只是设置访问表态文件对应的url。
静态文件存放的物理目录，是由SATICFILES_DIRS = [os.path.join(BASE_DIR, "static")] 来设置的。




【设置cookie】
def set_cookie(request):
    response = HttpResponse("设置cookie")
    response.set_cookie("num", 1) # 关了浏览器就没了
    response.set_cookie("num", 1, max_age = 14 * 24 * 3600) # 多少秒之后过期
    response.set_cookie("num", 1，expires = datetime.now()+timedelta(days=14)) # 14天之后过期
    return response


【读取cookie】
def get_cookie(request):
    num = reuest.COOKIES["num"]
    return HttpResponse(num)


【设置session】
def set_session(request):
    request.session['username'] = 'smart'
    request.session['age'] = 18
    return HttpResponse("set session")


【读取session】
def get_session(request):
    username = request.session['username']
    age = request.session['age']
    # request.session.get("key", 默认值)
    return HttpResponse("{}{}".format(username,age))

request.session.clear() # 清除所有session，在存储中删除值部分
request.session.flush() # 清除session数据，在存储中删除session的整条数据。
del request.session["key"] # 删除session中指定的键值，在存储中只删除某个键及对应的值。

request.session.has_key('key') # 检查是否有某个key

request.session.set_expiry(value) # 设置会话过期时间，没有指定就是两星期后过期
如果value是一个整数，会话的session_id cookie将在 value秒没有活过后过期
如果value为0，那么用户会话的session_id cookie将在用户关闭浏览器时过期
如果valueo None，那么会话的session_id cookie两击后过期


【模板变量】
模板变量名是由数字、字母、下划线和点组成，不能以下划线开头。
模板变量的解析顺序：
{book.btile}
1、首先把book当成一个字典，把btitle当成键名，进行取值book['btitle']
2、首先把book当成一个对象，把btitle当成属性，进行取值book.btitle
3、首先把book当成一个字典，把btitle当成对象的方法，进行取值book.btitle

如果解析失败，则产生内容时用空字符串填充模板变量。




【模板标签】
{% 代码段 %}
for 循环
{% for x in list %}
{% empty %} # 列表为空时执行
{% endfor %}
{{forloop.counter}} 得到for循环遍历到了第几次，从1开始

{% if 条件 %}
{% elif 条件 %}
{% else %}
{% endif %}
关系比较操作 > >= < <= == !=
注意：进行比较操作时，比较操作符两边必须有空格
逻辑运算符：not and or


【过滤器】---用于对模板变量进行操作，至少有一个参数，最多两个，其它个数不行。
| 左边的变量作为过滤器的第一个参数传给过滤器方法，：右边的变量作为过滤器的第二个参数。
date：改变日期的显示格式  如：{{ book.bpub_date|date:"Y年-m月-d日" }}
length：求长度，字符串，列表，元组，字典
default：设置模板变量的默认值。
escapte：转义HTML符号。

格式：模板变量 | 过滤器:参数
内容过滤器有很多，需要使用时查询文档


【自定义过滤器】---过滤器的本质就是python函数
1、在应用的目录下建一个【包】，名字叫 templatetags,名字不能写错
2、在包目录下建python文件，文件名随意
from django.template import Library
your_filters = Library()

@your_filter # 装饰你写的方法
def mod(num):
    return num % 2 == 0


3、在模板文件中加载过滤器所在的文件，就可以使用自定义的过滤器了
{{% load filter_file %}}




【模板继承】
1、建一个父模板，如：base.html
2、在子模板中(一开头)继承你模板 {% extends "base.html" %}
3、父模板中，需要被子模板替换的地方，用{% block 块名 %} 中间这里可写可不写 {% end blok 块名 %} 声明
4、在子模板中， {% block 块名 %} 重写的内容 {% end blok 块名 %}
5、还可以先取父模板中内容，再重写
{% block 块名 %}
{{ block.super }} # 获取父模板中的内容
重写的内容 
{% end blok 块名 %}


【html转义】
如果视图中传递的变量，有html标签的符号，默认是会被转义掉的，如传递 {"content":"<h1>转义</h1>"}，转义后的内容会原样显示，不会当作html
如果想要关闭html转义，可以用 save 过滤器关闭转义(针对一个变量)
也可以 {% autoescape off|on%} 模板语言代码 {% endautoescapte %} 来关闭或开启转义(针对一块代码)



【登录验证】一般用装饰器来处理登录后才能访问的页面，然后在需要检查的视图方法上装饰
def login_requried(view_func):
    def wrapper(request, *args, **kwargv):
        if request.sssion.has_key("islogin"):
            return view_func(request,*argv, **kwargv)
        else:
            return redirect('/login')
    return wrapper

【关于跨站攻击csrf】
django默认已开启防csrf攻击，在项目的setting文件中，MIDDLEWARE_CLASSES中。
只针对post提交。所以，在<form>表单内在加上一句 {% csrf_token %}，否则会显示403错误。



【url反向解释】
使用场景：一般一个网页在很多页面都可以回到首页，假设是index.html，如果首页的地址改成了new_index.html，那么，所有写了index.html的地方
都得改过来。
django提供burl反向解释的功能：
1、在项目的url.py中urlpatterns中指定应用的命名空间，一般与应用保持一致
    ...
    url(r'^', include('booktest.urls', namespace='booktest'))

2、在应用的url.py文件的urlpatterns中给要反向解释的url规则加上一个 name 参数
    url('^new_index', views.index, name='index')

3、在模板文件中通过反向解释
  <a href='{% url 'booktext:index' %}'>首页</a>

反向解释有位置参数的地址  /show_args/1/2   url规则：url(r'^show_args/(\d+)/(\d+)', views.show_args, name='show_args')
   <a href='{% url 'booktest: show_args' 1 2 %}'>show_args/1/2</a>
  

反向解释有关键字参数的地址 /show_kwargv/c/d  
   url规则：url(r'^show_kwargv/(?P<c>\d+)/(?P<d>)\d+', views.show_kwargvs, name='show_kwargv')
   <a href='{% url 'booktest: show_argv' c=3 d=4 %}'>show_kwargs/3/4</a>
   

视图中反向解释:假设生重定向到index
from django.core.urlresolvers import reverse
def test_redirect(request):
    url = reverse("booktesxt:index")
    # 位置参数： url = reverse('booktest:show_args', args=(1, 2))  /show_args/1/2
    # 关键字参数：url = reverse('booktest:show_kwargs', kwargs={'c':3, 'd':4})
    return redirect(url)





【在视图文件中访问项目settings文件里面的配置】
from django.conf import settings

# setting.BASE_DIR
# ......



【中间件】
中间件函数是 django 框架预留的函数接口，可以用来干预请求和应答的过程。
中间件函数有：
__init__(self)：服务器响应 第一个 请求的进修调用。

process_request(self, request)：在产生request对象，进行url匹配之前调用。

process_view(self, request, func, *args, **kwargs)：是url匹配之后，调用视图方法之前调用。

process_response(self, request, response)：视图方法调用之后，返回内容给浏览器之前调用。

process_exception(self, request, exception)：视图方法出现异常会调用这个函数。如果注册的多个中间件类中包含这个函数的时，
调用的顺序与注册的顺序是【相反】的。

例如：禁止某个IP地址访问网站
# 获取客户端的IP地址： request.Meta['REMOTE_ADDR']

1、在应用下面新建一个middleware.py文件
2、将中间件函数放在一个类中作为实例方法，类名可以随意，一般是XXXMiddleware

from django.http import HttpResponse


class BlockedIPsMiddleware(object):
    """中间件类"""
	EXCLUDE_IPS = ['your ip address']

    def process_view(self, request, func, *args, kwargs):
     """中间件函数"""
		user_ip = request.META['REMOTE_ADDR']
		if user_ip in BlockedIPsMiddleware.EXCLUDE_IPS:
			returnm HttpResponse('<h1>Forbidden</h1>')

3、在项目的配置文件中注册你的类

MIDDLEWARE_CLASSES = {
	...
	'booktest.middleware.BlockIPsMiddleware',
}



【admint管理】

1、本地化语言、时区
2、创建超级管理员 python manage.py createsuperuser
3、注册模型类
4、自定义管理页面
   自定义模型管理类
   注册模型类的时候给 register 函数添加第二个参数，就是自定义模型管理类的名字。

模型管理类的一些属性：
list_per_page = 10 # 指定每页显示10条数据

list_display = ['field','func_name'] # 指定显示的属性(字段)，也可以写模型类里的方法名(显示的方法的返回值)
默认方法的返回值在页面上是没有排序功以的，如果要加上排序，一般在方法后面(与方法同级)func_name.admin_order_field = 'btitle' 指定根据什么字段排序。如果还想指定对应的列标题，可以在后面加上一句func_name.short_description = '表头'，一些关联的数据简单显示一下，可以用方法来做。

模型类的属性想要显示中文表示，可在定义时加一个verbose_name参数
btitle = models.CharField(verbose_name='表头', max_length=20)

actions_on_top = True|False # 开启或关闭列表头部的动作

actions_on_bottom = True|False #开启或关闭列表底部的动作

list_filter = ['btitle'] # 列表右侧的过滤栏

search_fields = ['btitle'] # 列表上面的搜索框

fields = ['field_name', ...] # 调整模型编辑页面显示的顺序

fieldsets = (('基本', {'fields':['atitle']}), ...) # 分组显示，与fields互斥，只要选其中一个


【显示关联对象】
在一对多的头系中，可以在一端的编辑页面中编辑多端的对象，嵌入多端对象的方式包括表格、块两种。
类型InlineModelAdmin：表示在模型的编辑页面嵌入关联模型的编辑。
子类 TabularInline：以表格的形式嵌入。
字类 StackedInline：以块的形式嵌入。

1、打开booktest/admin.py文件，创建 AreaStackInline类

# 块
class AreaStackedInline(admin.StackedInline):
	model = AreaInfo # 要关系的对象，即多类的类名
	extra = 2 # 一个块中显示的个数


class AreaTabularInline(admin.TabularInline):
	model = AreaInfo
	extra = 2


class AreaAdmin(admin.ModelAdmin):  # 模型的管理类，不是模型类
	...
	inlines = [AreaStackedInline]
	# inlines =[AreaTabularInline] # 表格嵌入



【重写django管理页面】
管理页的模板，在django的包里面可以找到
.../site-pakages/django/contrib/admin/templates/admin/base_site.html
可以复制到项目的templates/admin(自己建)/中，然后进行修改


【上传图片】
1、在项目目下建一个目录存放图片(这里放在static下面)
2、到项目的setting文件中配置上传文件保存路径
MEDIA_ROOT = os.path.join(BASE_DIR, 'static/media')

上传图片分为两种使用场影：1，后台管理页面上传图片；2，用户自己上传图片；

【后台管理页面上传图片】
1、设计模型类（在model文件中）
class PicTest(model.Model):
	gpic = model.ImageField(upload_to='booktest') # 指定上传目录，相当于 /static/media/booktest，booktest要自己先建


2、迁移生成数据库表
3、注册模型类



【用户上传图片】
1、定义用户上传图片的页面(是一个自定义的表单)
<form method='post' action='/upload_handle' enctype='mutipart/form-data'>
	{% csrf_token %}
	<input type='file' name='pic' /><br/>
	<input type='submit' value='上传' /><br/>
</form>

method一定是post
entype一定是multipar/form-data
action是你声明的处理上传文件的视图

2、定义一个视图，保存上传的文件

def upload_handle(request):
	# 1 获取上传文件的处理对象
	pic = request.FILES['pic'] # 表单中上传文件的框的 name 是什么，这里就取什么
	# 如果上传的文件 不大于 2.5M，pic就是MemoryFileUploadHandler类型，否则是TemporaryFileUploadHander类型
	# pic.name 文件名字
	# pic.chunks() 是一个生成器

	# 2 创建一个文件(访问配置要先导致配置 from django.conf import settings)
	save_path = '%s/booktest/%s' % (settings.MEDIA_ROOT, pic.name)
	with open(save_path, 'wb') as f:
		# 3 获取上传文件的内容并写到创建的文件中
		for content in pic.chunks():
			f.write(content)

	# 4 在数据库中保存上传记录
	PicTest.objects.create(gpic='booktest/%s' & pic.name) #这里假设上传的文件是图片，就使用了PicTest类
	# 5 返回
	return HttpResponse("ok")





【分页】--点击 第 i 页的时候，变显示第 i 页的数据
from django.core.paginator import Paginator

def show_all_books(request, index):
    books = BookInfo.objects
    paginator = Paginator(books, 10) # 一页10条数据
    # 注意传过来的index是字符器，要转int
    if index == "":
        index = 1
    else:
        index = int(index)
        
    page = paginator.page(index) # page是Page类的实例
    return render(request, 'booktest/show_all_books.html', {'page': page})


Paginator类的属性：
num_pages：返回分布之后的总页数
page_range：返回分页后页码的列表

Paginator类的方法：
page(number)：返回第 number 页的 Page 类的实例

Page类的属性：
number：返回当前页的页码
object_list：返回包含当前页的数据
paginator：返回对应的 Paginator类对象

Page类的方法：
has_previous：判断当前页是否有前一页
has_next：判断当前页是否有下一页
previous_page_number：返回前一页的页码，不存在前一页会异常
next_page_number：返回后一页的页码，不存在后一页会异常
has_other_pages：如果有上一页或下一页，返回True
start_index：返回当前页上的第一个对象，相对于分页列表的所有对象的序号，从1开始。如将5个对象的列表分为每页2个，那么第2页的start_index 为3
end_index：返回当前页上的最后一个以对象。


前端使用：

# {% for book in page.object_list %}
{{% for book in page %}} # 因为page对象是可迭代的，所以直接遍历page也是可以的
    <li>{{ book.btitle}}</li>
{% endfor %}

{% if page.has_previous %}
    <a href='/show_books{{page.previous_page_number}}'>&lt;上一页</a>
{% endif %}

{% for index in page.paginator.page_range %}
    {% if index == page.number %}
        {{index}} #当前页不是超链接
    {% else %}
        <a href='/show_books{{index}}'>{{index}}</a>
    {% endif %}
{% endfor %}

{% if page.has_next %}
    <a href='/show_books{{page.next_page_number}}'>&gt;下一页</a>
{% endif %}




"""